AF 12-2023

# Math With Bad Drawings: Twin Cities

From the Math with Bad Drawings post [Data Science Contest: Which U.S. Cities are the True Twins?](https://mathwithbaddrawings.com/2023/12/04/data-science-contest-which-u-s-cities-are-the-true-twins/)

> Now, here comes the puzzle. It’s a two-part challenge:
> 
> * Find a complete list of all pairs of U.S. cities that meet this definition (at most 10 miles apart, with at least 200,000 people each, and populations within a factor of two).
> * From this list of twin cities, make a cogent and persuasive case for which pair deserves to be called THE twin cities.



In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from libpysal import weights

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# when we calculate distances, our projection is going to be in meters. Let's convert that now. 
MILES = 10
METERS = MILES * 1609.344

## Part 1: Find a complete list of all pairs of U.S. cities that meet this definition (at most 10 miles apart, with at least 200,000 people each, and populations within a factor of two)

In [2]:
# read in gazetteer data
place = pd.read_csv("data/2022_Gaz_place_national.txt", sep = "\t", dtype = {'GEOID': str}).iloc[:,[1,3,-2,-1]]
place.columns = ['GEOID', 'NAME', 'LAT', 'LONG']
place.head()

,GEOID,NAME,LAT,LONG
0,0100100,Abanda CDP,33.091627,-85.527029
1,0100124,Abbeville city,31.564706,-85.259121
2,0100460,Adamsville city,33.602315,-86.971527
3,0100484,Addison town,34.202681,-87.178004
4,0100676,Akron town,32.879066,-87.740899


In [3]:
pop_cols = ['SUMLEV', 'STATE', 'PLACE', 'NAME', 'STNAME', 'POPESTIMATE2022']

# read in population data
pop = pd.read_csv("data/sub-est2022.csv", dtype = {'STATE': str, 'PLACE': str})[pop_cols]
print(pop.shape)

# only keep incorporated places
ip = pop[pop.SUMLEV==162]

# add in GEOID as ID
ip['GEOID'] = ip['STATE'] + ip['PLACE']
ip['PLACE_NAME'] = ip['NAME'] + ", " + ip['STNAME']

print(ip.shape)

(81395, 6)
(19493, 8)


/var/folders/cs/6tsmkts96lqgd7z7tz4b521sl4vb6k/T/ipykernel_62073/3636733045.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip['GEOID'] = ip['STATE'] + ip['PLACE']
/var/folders/cs/6tsmkts96lqgd7z7tz4b521sl4vb6k/T/ipykernel_62073/3636733045.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip['PLACE_NAME'] = ip['NAME'] + ", " + ip['STNAME']


### At least 200k people each

In [4]:
ip_200k = ip[ip.POPESTIMATE2022>=200000]

print(ip_200k.shape)
ip_200k.head()

(125, 8)


,SUMLEV,STATE,PLACE,NAME,STNAME,POPESTIMATE2022,GEOID,PLACE_NAME
216,162,01,37000,Huntsville city,Alabama,221933,0137000,"Huntsville city, Alabama"
1121,162,02,03000,Anchorage municipality,Alaska,287145,0203000,"Anchorage municipality, Alaska"
1472,162,04,12000,Chandler city,Arizona,280711,0412000,"Chandler city, Arizona"
1490,162,04,27400,Gilbert town,Arizona,275346,0427400,"Gilbert town, Arizona"
1491,162,04,27820,Glendale city,Arizona,252136,0427820,"Glendale city, Arizona"


In [5]:
# do a left merge of ip_200k and the place data
ip_200k_merge = pd.merge(ip_200k, place, on = 'GEOID', how = 'left', validate = "1:1", indicator=True)
print(ip_200k_merge._merge.value_counts())
ip_200k_merge.head()

_merge
both          125
left_only       0
right_only      0
Name: count, dtype: int64


,SUMLEV,STATE,PLACE,NAME_x,STNAME,POPESTIMATE2022,GEOID,PLACE_NAME,NAME_y,LAT,LONG,_merge
0,162,01,37000,Huntsville city,Alabama,221933,0137000,"Huntsville city, Alabama",Huntsville city,34.782275,-86.532599,both
1,162,02,03000,Anchorage municipality,Alaska,287145,0203000,"Anchorage municipality, Alaska",Anchorage municipality,61.174250,-149.284329,both
2,162,04,12000,Chandler city,Arizona,280711,0412000,"Chandler city, Arizona",Chandler city,33.282765,-111.851819,both
3,162,04,27400,Gilbert town,Arizona,275346,0427400,"Gilbert town, Arizona",Gilbert town,33.310225,-111.743224,both
4,162,04,27820,Glendale city,Arizona,252136,0427820,"Glendale city, Arizona",Glendale city,33.533111,-112.189901,both


In [6]:
# clean up this dataset 
useful_cols = ['GEOID', 'PLACE_NAME', 'POPESTIMATE2022', 'LAT', 'LONG']
rename = ['GEOID', 'PLACE_NAME', 'POP', 'LAT', 'LONG']

processed = ip_200k_merge[useful_cols]
processed.columns = rename

# convert the lat and longs to a geodataframe
gdf = gpd.GeoDataFrame(processed, geometry=gpd.points_from_xy(processed['LONG'], processed['LAT'])).set_crs("EPSG:4326").to_crs("EPSG:5070")
gdf.head()

,GEOID,PLACE_NAME,POP,LAT,LONG,geometry
0,0137000,"Huntsville city, Alabama",221933,34.782275,-86.532599,POINT (857968.080 1345214.881)
1,0203000,"Anchorage municipality, Alaska",287145,61.174250,-149.284329,POINT (-3061965.373 5052524.860)
2,0412000,"Chandler city, Arizona",280711,33.282765,-111.851819,POINT (-1460085.669 1256914.279)
3,0427400,"Gilbert town, Arizona",275346,33.310225,-111.743224,POINT (-1449669.561 1258275.463)
4,0427820,"Glendale city, Arizona",252136,33.533111,-112.189901,POINT (-1486187.633 1289714.634)


### At most 10 miles apart

In [7]:
# calculate all pairs that are 10 miles or less from each other, keeping in duplicates (we will drop them later)
adj = weights.DistanceBand.from_dataframe(gdf, threshold=METERS, ids = 'PLACE_NAME', silence_warnings=True).to_adjlist(False, True)[['focal', 'neighbor']]

print(f"Number of (double counted) pairs: {len(adj)}")
adj

Number of (double counted) pairs: 26


,focal,neighbor
0,"Chandler city, Arizona","Gilbert town, Arizona"
1,"Gilbert town, Arizona","Chandler city, Arizona"
2,"Gilbert town, Arizona","Mesa city, Arizona"
3,"Glendale city, Arizona","Phoenix city, Arizona"
4,"Mesa city, Arizona","Gilbert town, Arizona"
5,"Phoenix city, Arizona","Glendale city, Arizona"
6,"Fontana city, California","San Bernardino city, California"
7,"Irvine city, California","Santa Ana city, California"
8,"San Bernardino city, California","Fontana city, California"
9,"Santa Ana city, California","Irvine city, California"


In [8]:
# create dictionary of city populations
pop = gdf[['PLACE_NAME', 'POP']].set_index('PLACE_NAME').to_dict()['POP']

# add in population information 
adj['focal_pop'] = adj['focal'].map(pop)
adj['neighbor_pop'] = adj['neighbor'].map(pop)

# calculate ratio of populations
adj['ratio'] = adj['focal_pop']/adj['neighbor_pop']

### Populations within a factor of 2

In [9]:
# keep all pairs with the right ratio.
# Since we kept in duplicates when we created the list of pairs, we don't have to
# include pairs with ratios between 0.5 and 1
all_twins = adj[(adj.ratio<=2) & (adj.ratio>=1)]

print("Final for part 1: Twin cities candidates")
all_twins.sort_values('ratio')


Final for part 1: Twin cities candidates


,focal,neighbor,focal_pop,neighbor_pop,ratio
7,"Irvine city, California","Santa Ana city, California",313685,308189,1.017833
0,"Chandler city, Arizona","Gilbert town, Arizona",280711,275346,1.019485
8,"San Bernardino city, California","Fontana city, California",220328,212475,1.036960
21,"Frisco city, Texas","McKinney city, Texas",219587,207507,1.058215
18,"Newark city, New Jersey","Jersey City city, New Jersey",305344,286670,1.065141
25,"Plano city, Texas","Frisco city, Texas",289547,219587,1.318598
14,"Minneapolis city, Minnesota","St. Paul city, Minnesota",425096,303176,1.402143
11,"Denver city, Colorado","Aurora city, Colorado",713252,393537,1.812414
4,"Mesa city, Arizona","Gilbert town, Arizona",512498,275346,1.861287
20,"Arlington city, Texas","Grand Prairie city, Texas",394602,201843,1.954995


## Part 2: From this list of twin cities, make a cogent and persuasive case for which pair deserves to be called THE twin cities.

We have 10 potential pairs of candidates! Time to narrow things down. As a reminder, these pairs are: 

**Potential twin cities** 

| focal                        | neighbor | focal_pop | neighbor_pop | ratio |
| ---------------------------- | -------- | --------- | ------------ | ----- | 
| Irvine city, California         | Santa Ana city, California   | 313685   | 308189    | 1.017833     |
| Chandler city, Arizona          | Gilbert town, Arizona        | 280711   | 275346    | 1.019485     |
| San Bernardino city, California | Fontana city, California     | 220328   | 212475    | 1.036960     |
| Frisco city, Texas              | McKinney city, Texas         | 219587   | 207507    | 1.058215     |
| Newark city, New Jersey         | Jersey City city, New Jersey | 305344   | 286670    | 1.065141     |
| Plano city, Texas               | Frisco city, Texas           | 289547   | 219587    | 1.318598     |
| Minneapolis city, Minnesota     | St. Paul city, Minnesota     | 425096   | 303176    | 1.402143     |
| Denver city, Colorado           | Aurora city, Colorado        | 713252   | 393537    | 1.812414     |
| Mesa city, Arizona              | Gilbert town, Arizona        | 512498   | 275346    | 1.861287     |
| Arlington city, Texas           | Grand Prairie city, Texas    | 394602   | 201843    | 1.954995     |

Here is where I should probably note that this isn't the first dataset that I tried using when I first did this. Previously, I had also used the US cities database from [Simplemaps](https://simplemaps.com/data/us-cities), which led me to a different final list: 

**Simplemaps list of twin cities**

| focal | neighbor | focal_pop | neighbor_pop | ratio |
| --- | --- | --- | --- | --- |
| Enterprise, NV | Spring Valley, NV | 219566 | 217441 | 1.009773 |
| Chandler, AZ | Gilbert, AZ | 272439 | 262249 | 1.038856 |
| Santa Ana, CA | Irvine, CA | 313818 | 297868 | 1.053547 |
| San Bernardino, CA | Fontana, CA | 220821 | 208087 | 1.061196 |
| Newark, NJ | Jersey City, NJ  | 306247 | 287146 | 1.066520 |
| Anaheim, CA | Santa Ana, CA | 348204 | 313818 | 1.109573 |
| Mesa, AZ | Gilbert, AZ | 497752 | 262249 | 1.898013 |

As you can see, Minneapolis and St. Paul aren't on this list, which led me to switch datasets. (I think it's because the simplemaps database pulled metropolitan area data for Minneapolis, which gave it a much larger population than St. Paul. The dataset also included Enterprise, NV and Spring Valley, NV, both of which are unincorporated towns and therefore not cities.)

Regardless, I included this because my first criteria for the twin cities is: 

### 1. No twin city should have a city that could be a twin city with another city. 

If a city can be twins with two different cities, that's a complicated mix between half-brothers and triplets. Definitely not twin cities. 

For this criteria, I will also be including the simplemaps analysis, because I don't think there should be any confusion based on the dataset. This removes the following cities: 
* Gilbert, Arizona (which is a town anyway)
* Santa Ana, California
* Frisco, Texas 

which in turn removes the following pairs from the running: 

| focal                        | neighbor | focal_pop | neighbor_pop | ratio |
| ---------------------------- | -------- | --------- | ------------ | ----- | 
| Irvine city, California         | Santa Ana city, California   | 313685   | 308189    | 1.017833     |
| Chandler city, Arizona          | Gilbert town, Arizona        | 280711   | 275346    | 1.019485     |
| Frisco city, Texas              | McKinney city, Texas         | 219587   | 207507    | 1.058215     |
| Plano city, Texas               | Frisco city, Texas           | 289547   | 219587    | 1.318598     |
| Mesa city, Arizona              | Gilbert town, Arizona        | 512498   | 275346    | 1.861287     |

Leaving: 

| focal                        | neighbor | focal_pop | neighbor_pop | ratio |
| ---------------------------- | -------- | --------- | ------------ | ----- | 
| San Bernardino city, California | Fontana city, California     | 220328   | 212475    | 1.036960     |
| Newark city, New Jersey         | Jersey City city, New Jersey | 305344   | 286670    | 1.065141     |
| Minneapolis city, Minnesota     | St. Paul city, Minnesota     | 425096   | 303176    | 1.402143     |
| Denver city, Colorado           | Aurora city, Colorado        | 713252   | 393537    | 1.812414     |
| Arlington city, Texas           | Grand Prairie city, Texas    | 394602   | 201843    | 1.954995     |


### 2. The population ratio should be less than 1.5.

Next, to narrow down, it's clear that between Denver and Aurora, Denver is the larger, more prominent city. I'm going to make the population ratio restriction even stricter to 1.5, meaning that one city has to be less than 50% larger than the other city.

This removes: 

| focal                        | neighbor | focal_pop | neighbor_pop | ratio |
| ---------------------------- | -------- | --------- | ------------ | ----- | 
| Denver city, Colorado           | Aurora city, Colorado        | 713252   | 393537    | 1.812414     |
| Arlington city, Texas           | Grand Prairie city, Texas    | 394602   | 201843    | 1.954995     |


Leaving: 

| focal                        | neighbor | focal_pop | neighbor_pop | ratio |
| ---------------------------- | -------- | --------- | ------------ | ----- | 
| San Bernardino city, California | Fontana city, California     | 220328   | 212475    | 1.036960     |
| Newark city, New Jersey         | Jersey City city, New Jersey | 305344   | 286670    | 1.065141     |
| Minneapolis city, Minnesota     | St. Paul city, Minnesota     | 425096   | 303176    | 1.402143     |
 
Three pairs left!

### Final winner: Newark, New Jersey and Jersey City, New Jersey

At this point, my final criteria was that the names needed to sound like cute twin names. 

Then this conversation with New Jersey couldn't escape my head: 

> **Doctor:** Hey New Jersey, you're going to have twins! So exciting! What are you going to name them? 
>
> **New Jersey:** Uh...I wasn't prepared for this. (New new jersey no longer works!) Hm...let's just name one of them after "New" and the other one after "Jersey"
>
> *Newark proceeds to be born first and is slightly bigger and therefore gets the first part of the name. Jersey City follows shortly. For the rest of their life, Newark will constantly brag about the fact that they're 12 minutes older to Jersey City. They will also brag about having an international airport.*
> 
> *Both Newark and Jersey City will grow up realizing that they were constantly in the shadow of their older, more famous cousin New York City, New York. They will bond over this fact.* 

There we have it. The revised twin cities!